In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", 101)
import os
import cv2
import json
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["font.size"] = 15
from collections import Counter
from PIL import Image
import math
from collections import defaultdict
from pathlib import Path
import cv2
from tqdm import tqdm
import seaborn as sns

In [2]:
# path = './data/'

# data = pd.read_csv(path + 'train.csv')
# data['ClassId'] = data['ClassId'].astype(int)
# data = data.pivot(index='ImageId',columns='ClassId',values='EncodedPixels')

# data.head()

In [3]:
# class_dict = defaultdict(int)

# kind_class_dict = defaultdict(int)

# no_defects_num = 0
# defects_num = 0

# for i in range(0, len(data)):
#     img_names = [data.iloc[i, 0]]
        
#     labels = data.iloc[i, 0:4]

#     if labels.isna().all():
#         no_defects_num += 1
#     else:
#         defects_num += 1
    
#     kind_class_dict[sum(labels.isna().values == False)] += 1
        
#     for idx, label in enumerate(labels.isna().values.tolist()):
#         if label == False:
#             class_dict[idx+1] += 1

In [4]:
# print("the number of images with no defects: {}".format(no_defects_num))
# print("the number of images with defects: {}".format(defects_num))

In [5]:
# fig, ax = plt.subplots()
# sns.barplot(x=list(class_dict.keys()), y=list(class_dict.values()), ax=ax)
# ax.set_title("the number of images for each class")
# ax.set_xlabel("class")
# class_dict

In [6]:
# fig, ax = plt.subplots()
# sns.barplot(x=list(kind_class_dict.keys()), y=list(kind_class_dict.values()), ax=ax)
# ax.set_title("Number of classes included in each image");
# ax.set_xlabel("number of classes in the image")
# kind_class_dict

In [7]:
# train_size_dict = defaultdict(int)
# train_path = Path("./data/train_images/")

# for img_name in train_path.iterdir():
#     img = Image.open(img_name)
#     train_size_dict[img.size] += 1

In [8]:
# test_size_dict = defaultdict(int)
# test_path = Path("./data/test_images/")

# for img_name in test_path.iterdir():
#     img = Image.open(img_name)
#     test_size_dict[img.size] += 1

In [9]:
# path = './data/'

# data = pd.read_csv(path + 'train.csv')
# data.head()

In [10]:
# data = data[data['EncodedPixels'].notnull()].reset_index(drop=True)
# print(len(data))
# data.head()

In [11]:
def make_mask(row_id, df):
    '''Given a row index, return image_id and mask (256, 1600, 4) from the dataframe `df`'''
    fname = df.iloc[row_id].name
    labels = df.iloc[row_id][:4]
    masks = np.zeros((256, 1600, 4), dtype=np.float32)

    for idx, label in enumerate(labels.values):
        if label is not np.nan:
            label = label.split(" ")
            positions = map(int, label[0::2])
            length = map(int, label[1::2])
            mask = np.zeros(256 * 1600, dtype=np.uint8)
            for pos, le in zip(positions, length):
                mask[pos:(pos + le)] = 1
            masks[:, :, idx] = mask.reshape(256, 1600, order='F')
    return fname, masks

In [12]:
from albumentations import HorizontalFlip, Normalize, Resize, Compose
from albumentations.pytorch import ToTensor

def get_transforms(phase):
    list_transforms = []
#     if phase == "train":
#         list_transforms.extend(
#             [
#                 HorizontalFlip(p=0.5), # only horizontal flip as of now
#             ]
#         )
    list_transforms.extend(
        [
            Resize(256, 256),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], p=1),
            ToTensor(),
        ]
    )
    list_trfms = Compose(list_transforms)
    return list_trfms

In [13]:
from torch.utils.data import Dataset

class SteelDataset(Dataset):
    def __init__(self, df, data_folder, phase):
        self.df = df
        self.root = data_folder
        self.phase = phase
        self.transforms = get_transforms(phase)
        self.fnames = self.df.index.tolist()

    def __getitem__(self, idx):
        image_id, mask = make_mask(idx, self.df)
        image_path = os.path.join(self.root, "train_images",  image_id)
        img = cv2.imread(image_path)
        augmented = self.transforms(image=img, mask=mask)
        img = augmented['image']
        mask = augmented['mask'] # 1x256x1600x4
        mask = mask[0].permute(2, 0, 1) # 4x256x1600
        return img, mask

    def __len__(self):
        return len(self.fnames)


In [14]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

def SteelDataLoader(
    data_folder,
    df,
    phase,
    batch_size=8,
    num_workers=4,
):
    '''Returns dataloader for the model training'''
    image_dataset = SteelDataset(df, data_folder, phase)
    dataloader = DataLoader(
        image_dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=True,
        shuffle=True,   
    )

    return dataloader


# data['ClassId'] = data['ClassId'].astype(int)
# data = data.pivot(index='ImageId',columns='ClassId',values='EncodedPixels')

In [15]:
# train_df, val_df = train_test_split(data, test_size=0.2, stratify=data.count(axis=1), random_state=69)
# train_loader = SteelDataLoader("./data/", train_df, "train", 4, 4)
# val_loader = SteelDataLoader("./data/", val_df, "val", 4, 4)

In [16]:
train_df = pd.read_csv("./data/new_train.csv", index_col="ImageId")
val_df = pd.read_csv("./data/new_test.csv", index_col="ImageId")
train_loader = SteelDataLoader("./data/", train_df, "train", 4, 4)
val_loader = SteelDataLoader("./data/", val_df, "val", 4, 4)

In [17]:
train_df.head()

,1,2,3,4
ImageId,,,,
ca492d6f1.jpg,NaN,NaN,154203 90 154370 179 154626 179 154882 179 155...,NaN
8b495e389.jpg,NaN,NaN,258819 10 259075 30 259331 49 259587 69 259843...,NaN
ebffb5d31.jpg,NaN,NaN,14420 5 14676 14 14932 22 15187 28 15443 28 15...,NaN
ce22cd852.jpg,NaN,NaN,40462 1 40717 3 40971 5 41226 7 41481 8 41736 ...,NaN
24e3826ff.jpg,NaN,NaN,256509 4 256758 11 257006 19 257255 26 257507 ...,NaN


In [41]:
len(train_df)

5332

In [42]:
len(val_df)

1334

In [18]:
def predict(X, threshold):
    '''X is sigmoid output of the model'''
    X_p = np.copy(X)
    preds = (X_p > threshold).astype('uint8')
    return preds

In [19]:
def compute_dice(probability, truth, threshold=0.5, reduction='none'):
    '''Calculates dice of positive and negative images seperately'''
    '''probability and truth must be torch tensors'''
    batch_size = len(truth)
    with torch.no_grad():
        probability = probability.view(batch_size, -1)
        truth = truth.view(batch_size, -1)
        assert(probability.shape == truth.shape)

        p = (probability > threshold).float()
        t = (truth > 0.5).float()

        t_sum = t.sum(-1)
        p_sum = p.sum(-1)
        neg_index = torch.nonzero(t_sum == 0, as_tuple=False)
        pos_index = torch.nonzero(t_sum >= 1, as_tuple=False)

        dice_neg = (p_sum == 0).float()
        dice_pos = 2 * (p*t).sum(-1)/((p+t).sum(-1))

        dice_neg = dice_neg[neg_index]
        dice_pos = dice_pos[pos_index]
        dice = torch.cat([dice_pos, dice_neg])

        num_neg = len(neg_index)
        num_pos = len(pos_index)

    return dice, dice_neg, dice_pos, num_neg, num_pos


def compute_ious(pred, label, classes, ignore_index=255, only_present=True):
    pred[label == ignore_index] = 0
    ious = []
    for c in classes:
        label_c = label == c
        if only_present and np.sum(label_c) == 0:
            ious.append(np.nan)
            continue
        pred_c = pred == c
        intersection = np.logical_and(pred_c, label_c).sum()
        union = np.logical_or(pred_c, label_c).sum()
        if union != 0:
            ious.append(intersection / union)
    return ious if ious else [1]

def compute_iou_batch(outputs, labels, classes=None):
    ious = []
    preds = np.copy(outputs)
    labels = np.array(labels)
    for pred, label in zip(preds, labels):
        ious.append(np.nanmean(compute_ious(pred, label, classes)))
    iou = np.nanmean(ious)
    return iou

In [20]:
class Logger:
    '''A Logger to keep track of iou and dice scores throughout an epoch'''
    def __init__(self, phase, epoch):
        self.base_threshold = 0.5
        self.base_dice_scores = []
        self.dice_neg_scores = []
        self.dice_pos_scores = []
        self.iou_scores = []

    def update(self, targets, outputs):
        probs = torch.sigmoid(outputs)
        dice, dice_neg, dice_pos, _, _ = compute_dice(probs, targets, self.base_threshold)
        self.base_dice_scores.extend(dice.tolist())
        self.dice_pos_scores.extend(dice_pos.tolist())
        self.dice_neg_scores.extend(dice_neg.tolist())
        preds = predict(probs, self.base_threshold)
        iou = compute_iou_batch(preds, targets, classes=[1])
        self.iou_scores.append(iou)

    def get_metrics(self):
        dice = np.nanmean(self.base_dice_scores)
        dice_neg = np.nanmean(self.dice_neg_scores)
        dice_pos = np.nanmean(self.dice_pos_scores)
        dices = [dice, dice_neg, dice_pos]
        iou = np.nanmean(self.iou_scores)
        return dices, iou

def epoch_log(epoch, epoch_loss, meter):
    '''logging the metrics at the end of an epoch'''
    dices, iou = meter.get_metrics()
    dice, dice_neg, dice_pos = dices
    print("Loss: %0.4f | IoU: %0.4f | dice: %0.4f | dice_neg: %0.4f | dice_pos: %0.4f" % (epoch_loss, iou, dice, dice_neg, dice_pos))
    return dice, iou

In [21]:
import segmentation_models_pytorch as smp

model = smp.Unet("resnet18", encoder_weights="imagenet", classes=4, activation=None)

In [22]:
import torch

criterion = torch.nn.BCEWithLogitsLoss()

In [23]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=5e-3)

# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", patience=3, verbose=True)

In [24]:
# import time

# # define hyperparameters
# num_epochs = 20
# batch_size = 4
# accumulation_steps = 32 / batch_size
# best_loss = float("inf")

# train_losses = []
# train_iou = []
# train_dice = []
# valid_losses = []
# valid_iou = []
# valid_dice = []

# for epoch in range(num_epochs):
#     # Training
#     log = Logger("train", epoch)
#     start = time.strftime("%H:%M:%S")
#     print(f"Starting epoch: {epoch} | phase: train | ⏰: {start}")
#     model.train()
#     running_loss = 0.0
#     total_batches = len(train_loader)
#     optimizer.zero_grad()
#     for itr, batch in enumerate(train_loader):
#         images, targets = batch
#         outputs = model(images)
#         loss = criterion(outputs, targets)
#         loss = loss / accumulation_steps
#         loss.backward()
#         if (itr + 1 ) % accumulation_steps == 0:
#             optimizer.step()
#             optimizer.zero_grad()
#         running_loss += loss.item()
#         outputs = outputs.detach()
#         log.update(targets, outputs)
#     epoch_loss = (running_loss * accumulation_steps) / total_batches
#     dice, iou = epoch_log(epoch, epoch_loss, log)
#     train_losses.append(epoch_loss)
#     train_dice.append(dice)
#     train_iou.append(iou)
    
#     state = {
#         "epoch": epoch,
#         "best_loss": best_loss,
#         "state_dict": model.state_dict(),
#         "optimizer": optimizer.state_dict(),
#     }
    
#     # Validation
#     torch.no_grad()
#     val_log = Logger("val", epoch)
#     start = time.strftime("%H:%M:%S")
#     print(f"Starting epoch: {epoch} | phase: val | ⏰: {start}")
#     running_loss = 0.0
#     total_batches = len(val_loader)
#     optimizer.zero_grad()
#     for itr, batch in enumerate(val_loader):
#         images, targets = batch
#         outputs = model(images)
#         loss = criterion(outputs, targets)
#         loss = loss / accumulation_steps
#         running_loss += loss.item()
#         outputs = outputs.detach()
#         log.update(targets, outputs)
#     val_loss = (running_loss * accumulation_steps) / total_batches
#     dice, iou = epoch_log(epoch, epoch_loss, log)
#     valid_losses.append(val_loss)
#     valid_dice.append(dice)
#     valid_iou.append(iou)
    
#     scheduler.step(val_loss)
    
#     if val_loss < best_loss:
#         print("******** New optimal found, saving state ********")
#         state["best_loss"] = best_loss = val_loss
#         torch.save(state, "./model.pth")
#     print()
        
    

In [25]:
import time

# define hyperparameters
num_epochs = 30
batch_size = 4
accumulation_steps = 32 / batch_size
best_loss = float("inf")

train_losses = []
train_iou = []
train_dice = []
valid_losses = []
valid_iou = []
valid_dice = []

stop_patience = 4
epochs_no_improve = 0
early_stop = False
min_val_loss = np.Inf

for epoch in range(num_epochs):
    # Training
    log = Logger("train", epoch)
    start = time.strftime("%H:%M:%S")
    print(f"Starting epoch: {epoch} | phase: train | ⏰: {start}")
    model.train()
    running_loss = 0.0
    total_batches = len(train_loader)
    optimizer.zero_grad()
    for itr, batch in enumerate(train_loader):
        images, targets = batch
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss = loss / accumulation_steps
        loss.backward()
        if (itr + 1 ) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
        running_loss += loss.item()
        outputs = outputs.detach()
        log.update(targets, outputs)
    epoch_loss = (running_loss * accumulation_steps) / total_batches
    dice, iou = epoch_log(epoch, epoch_loss, log)
    train_losses.append(epoch_loss)
    train_dice.append(dice)
    train_iou.append(iou)
    
    state = {
        "epoch": epoch,
        "best_loss": best_loss,
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    
    # Validation
    torch.no_grad()
    val_log = Logger("val", epoch)
    start = time.strftime("%H:%M:%S")
    print(f"Starting epoch: {epoch} | phase: val | ⏰: {start}")
    running_loss = 0.0
    total_batches = len(val_loader)
    optimizer.zero_grad()
    for itr, batch in enumerate(val_loader):
        images, targets = batch
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss = loss / accumulation_steps
        running_loss += loss.item()
        outputs = outputs.detach()
        log.update(targets, outputs)
    val_loss = (running_loss * accumulation_steps) / total_batches
    dice, iou = epoch_log(epoch, epoch_loss, log)
    valid_losses.append(val_loss)
    valid_dice.append(dice)
    valid_iou.append(iou)
    
#     scheduler.step(val_loss)
    
    if val_loss < best_loss:
        print("******** New optimal found, saving state ********")
        torch.save(state, "./models/base-model.pth")
        state["best_loss"] = best_loss = val_loss
        epochs_no_improve = 0
        min_val_loss = val_loss
    else:
        epochs_no_improve += 1
        print("******** No improvement with patience: {}/{} ********".format(epochs_no_improve, stop_patience))
        
    if epochs_no_improve == stop_patience:
        print("Early Stopping")
        early_stop = True
        torch.save(state, "./models/base-model.pth")
        break
 
    print()
        
    

Starting epoch: 0 | phase: train | ⏰: 16:56:00


<ipython-input-19-930a3d73628a>:51: RuntimeWarning: Mean of empty slice
  ious.append(np.nanmean(compute_ious(pred, label, classes)))


Loss: 0.0715 | IoU: 0.0466 | dice: 0.0696 | dice_neg: 1.0000 | dice_pos: 0.0695
Starting epoch: 0 | phase: val | ⏰: 17:47:14
Loss: 0.0715 | IoU: 0.0610 | dice: 0.0910 | dice_neg: 1.0000 | dice_pos: 0.0909
******** New optimal found, saving state ********

Starting epoch: 1 | phase: train | ⏰: 17:52:06
Loss: 0.0453 | IoU: 0.1277 | dice: 0.1880 | dice_neg: 1.0000 | dice_pos: 0.1879
Starting epoch: 1 | phase: val | ⏰: 18:42:23
Loss: 0.0453 | IoU: 0.1362 | dice: 0.1999 | dice_neg: 1.0000 | dice_pos: 0.1998
******** New optimal found, saving state ********

Starting epoch: 2 | phase: train | ⏰: 18:47:18
Loss: 0.0432 | IoU: 0.1594 | dice: 0.2295 | dice_neg: 1.0000 | dice_pos: 0.2294
Starting epoch: 2 | phase: val | ⏰: 19:37:39
Loss: 0.0432 | IoU: 0.1556 | dice: 0.2255 | dice_neg: 1.0000 | dice_pos: 0.2254
******** New optimal found, saving state ********

Starting epoch: 3 | phase: train | ⏰: 19:42:33
Loss: 0.0409 | IoU: 0.1821 | dice: 0.2588 | dice_neg: 1.0000 | dice_pos: 0.2586
Starting ep

In [30]:
print(model)

Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [27]:
valid_losses

[0.047507763132006824,
 0.04511059701030037,
 0.044303568342176386,
 0.041112968449977166,
 0.0385437869650875,
 0.04159449860955516,
 0.037681647871961134,
 0.03539382869065433,
 0.03528842368737577,
 0.03514161287851542,
 0.03526779063308087,
 0.03374255122059209,
 0.03342576178688623,
 0.0333194823358431,
 0.03314540648699074,
 0.03249297933485694,
 0.030542568443064206,
 0.030157070533054584,
 0.03257794284490411,
 0.031795951366301874,
 0.030778167730275384,
 0.031842756246824463]

In [28]:
valid_dice

[0.09101097954120257,
 0.19988052570667536,
 0.22550082104115424,
 0.2556054148854991,
 0.2676964239269755,
 0.28853561944589895,
 0.31674041612954107,
 0.3369380512108265,
 0.32626062331855504,
 0.35343933165041663,
 0.35534717241748026,
 0.3686255076737534,
 0.3808251593846374,
 0.37695454563007474,
 0.3838130890114281,
 0.39235852733464177,
 0.4143753664173222,
 0.42095785286722376,
 0.4269684587552757,
 0.421779712117633,
 0.4460040663173385,
 0.44079903213397265]

In [29]:
valid_iou

[0.06102528609125538,
 0.13615440321076203,
 0.15559938926456066,
 0.17941115218958847,
 0.1884063216804733,
 0.20443758133042017,
 0.22582543564145113,
 0.2423667915343526,
 0.2355177015381089,
 0.2568795401136395,
 0.258266138554064,
 0.26994839672878607,
 0.28071899576504167,
 0.2772707721289887,
 0.2828826418781521,
 0.29111199575392616,
 0.31004365687295865,
 0.3154205501993534,
 0.3208358630056838,
 0.31647019265171517,
 0.33593251540422714,
 0.33165861999085156]

In [31]:
train_losses

[0.07150241661822425,
 0.04530455335619771,
 0.04324502807700223,
 0.040882852488104464,
 0.03970169169871613,
 0.038044158440566744,
 0.03685200896703786,
 0.035567230765762986,
 0.03468476925520345,
 0.03461148846469713,
 0.03289948402368097,
 0.032325752140969506,
 0.03064881803346302,
 0.03089100713873057,
 0.031192895439673504,
 0.029493485908572467,
 0.028696761093125453,
 0.028161130012561472,
 0.027349160990446514,
 0.027146287700154353,
 0.02628981644624463,
 0.025465117150181255]

In [32]:
train_dice

[0.06964268260120811,
 0.1880479630143968,
 0.2295215883880224,
 0.25877302967647214,
 0.2607196666150078,
 0.29615744127533583,
 0.3118908786151921,
 0.33340110724247973,
 0.34461766463116306,
 0.3459664844497932,
 0.36654389473659493,
 0.369510849049049,
 0.39071300662267117,
 0.3859582741417155,
 0.38813864291055084,
 0.4065985388015799,
 0.4127190244800668,
 0.41529690606393993,
 0.4254964952269529,
 0.4266719293075037,
 0.4447694511930573,
 0.4570935470813632]

In [33]:
train_iou

[0.04656693519237674,
 0.12768885173683056,
 0.1593569785621481,
 0.1821363143946487,
 0.1831146259456904,
 0.20991414677854967,
 0.22240878171802494,
 0.23950430381250423,
 0.24955058115312648,
 0.2506963054794647,
 0.26819265257144637,
 0.2704295176346331,
 0.28894094244982504,
 0.28511459079610496,
 0.2864017026798611,
 0.3029288965489734,
 0.3086857212945411,
 0.3106605263495961,
 0.31965546337508727,
 0.3204903980015809,
 0.3351293910839219,
 0.34545985060721346]

In [34]:
len(train_iou)

22